In [579]:
import os
import sys
from pathlib import Path
import pandas as pd

%matplotlib inline

In [580]:
sys.path.append(r"../LUCinSA_helpers")
from rf import *

In [581]:
'''
PARAMETERS: modify in notebook_params notebook, then run that notebook and this cell to update here
DO NOT modify this cell
'''

%store -r basic_config
print("Basic Parameters: \n time-series data is in (smooth_dir): {} \n"
      " modelling year is (filter_year param): {} (this is first year if season spans two years)"
      .format(basic_config['smooth_dir'], basic_config['filter_yr']))
%store -r classification_params
mod_type = "Max_nopp_nnweg5-9"
mod_name = "ratios_rebal"
classification_params['samp_pix_vars'] = '/home/downspout-cel/paraguay_lc/vector/tests/ptsgdb_{}.csv'.format(mod_type)
classification_params["samp_pts"] = '/home/downspout-cel/paraguay_lc/classification/RF/sample_dfs/{}.csv'.format(mod_name)

print("Classification_Params: \n" 
      " temp output files are saved to (local_model_dir): {} \n" 
      " shared modelling files are in (main_model_dir): {} \n" 
      " feature_model = {} \n sample_model = {} \n model_name = {} \n" 
      " model_type = {} \n samp_pt_file = {} \n pix_vars = {} \n samp_pts = {} \n lc_class = {} \n ranhold = {} \n impmeth = {}"
      .format(classification_params['local_model_dir'],classification_params['main_model_dir'],
              classification_params['feature_model'],classification_params['sample_model'],classification_params['model_name'],
              classification_params['model_type'],basic_config['ptfile'],
              classification_params['samp_pix_vars'],classification_params['samp_pts'],classification_params['lc_mod'],
              classification_params['ranhold'],classification_params['impmeth']))
      

Basic Parameters: 
 time-series data is in (smooth_dir): /home/downspout-cel/paraguay_lc/stac/grids 
 modelling year is (filter_year param): 2021 (this is first year if season spans two years)
Classification_Params: 
 temp output files are saved to (local_model_dir): /home/downspout-cel/paraguay_lc/vector/tests 
 shared modelling files are in (main_model_dir): /home/downspout-cel/paraguay_lc/classification 
 feature_model = Max 
 sample_model = testing 
 model_name = Max_testing_2021 
 model_type = RF 
 samp_pt_file = /home/downspout-cel/paraguay_lc/vector/sampleData/SamplePts_Dec2023_ALL.csv 
 pix_vars = /home/downspout-cel/paraguay_lc/vector/tests/ptsgdb_Max_nopp_nnweg5-9.csv 
 samp_pts = /home/downspout-cel/paraguay_lc/classification/RF/sample_dfs/ratios_rebal.csv 
 lc_class = All 
 ranhold = 29 
 impmeth = Impurity


## Merge dataframes together

In [582]:
lut=pd.read_csv('../Class_LUT.csv')
#print(lut.sort_values('LC_UNQ')[['LC_UNQ','USE_NAME','LC25','LC25_name']])

samp_pts = pd.read_csv(classification_params['samp_pts'])
#print(samp_pts.columns.tolist())
pix_vars = pd.read_csv(classification_params['samp_pix_vars'])
if mod_name == "base1000":
    samp_pts.rename(columns = {"Unnamed: 0": 'OID_'}, inplace = True)
#print(samp_pts)

pix_data = samp_pts.merge(pix_vars, left_on='OID_', right_on='OID_', how='inner')
#pix_data["smlhd_halfha"] = samp_pts["smlhd_halfha"]

#pix_data.drop(['LC2'], axis=1, inplace=True)
pixdf = pix_data.merge(lut, left_on='Class', right_on='USE_NAME', how='left')
print('sample breakdown by LC25 class:')
print(pixdf.isna().any().any())
print(pixdf['LC25_name_y'].value_counts())

print('default rf model with all sample pixels and pixel only data')

classification_params["pixdf"] = pixdf

print(classification_params["pixdf"]["smlhd_halfha"])
print(classification_params["pixdf"]["smlhld_1ha"])

#print(classification_params["pixdf"]["smlhd_halfha"].equals(classification_params["pixdf"]["smlhld_1ha"]))

for i in classification_params["pixdf"].columns.tolist():
    #print(i)
    if i == "smlhd_halfha":
        print("Yup, its there")


sample breakdown by LC25 class:
True
LC25_name_y
Crops-Soybeans       4486
Mixed-path           3898
Crops-mix            2981
Grassland-Managed    1858
Mixed-VegEdge        1826
Trees-Forest         1721
unknown              1686
Trees-disturbed      1170
Grassland-Wet        1060
Grassland-Natural    1054
TreePlant             838
Shrub                 800
Crops-Yerba-Mate      633
Crops-Rice            519
Grass_tree-mix        454
NoVeg_Water           408
NoVeg_Bare            401
Crops-Sugar           367
Crops-Banana          308
NoVeg_Built           285
Crops-Orchard         280
Cleared               262
Crops-Corn            243
Trees-Forest_palm     201
Burnt-woody            74
Name: count, dtype: int64
default rf model with all sample pixels and pixel only data
0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
27814    0.0
27815    0.0
27816    0.0
27817    0.0
27818    0.0
Name: smlhd_halfha, Length: 27819, dtype: float64
0        0.0
1        

## View the look up table
These are the different LC_models to group things in classification and to translate between numerical map categories and text labels

In [583]:
lut=pd.read_csv('../Class_LUT.csv')
lut.drop(['Description'], axis=1, inplace=True)

print(lut.sort_values('LC_UNQ'))

    LC_UNQ            USE_NAME          Segmentation  LCTrans  \
0        1               NoVeg                 NoVeg        1   
1        2          NoVeg_Bare                   NaN        2   
2        3         NoVeg_Built                   NaN        3   
3        7         NoVeg_Water                   NaN        7   
4        9          Mixed-path                   NaN        9   
5       10             Cleared                   NaN       10   
6       11       TreePlant-new                   NaN       11   
7       12   Grassland-Natural                   NaN       15   
8       13   Grassland-Managed  LowVeg_grass_managed       15   
9       15           Grassland          LowVeg_grass       15   
10      17       Grassland-Wet                   NaN       15   
11      18     Mixed-GrassEdge                   NaN       19   
12      19     Mixed-FieldEdge                   NaN       19   
13      20              LowVeg                LowVeg       20   
14      23  Crops-horticu

## get sample dataframe:
 pixdf is the combination of the sample point file and variable stack for those points (pix_vars).
 This is created in notebooks 6a and 6b.

In [584]:
"""print(classification_params['pixdf'])
pixdf = pd.read_csv(classification_params['pixdf'])
#pixdf = pixdf.dropna(inplace = True)
#print(pixdf)
#print(pixdf.isnull().any().any())
print('sample breakdown by {}:'.format(classification_params['lc_mod']))
label_col, new_lut = get_class_col(classification_params['lc_mod'], lut)
if '{}_name'.format(label_col) in pixdf.columns:
    print(pixdf['{}_name'.format(label_col)].value_counts())
else:
    pixdf2 = pixdf.merge(new_lut[['USE_NAME','{}'.format(label_col),'{}_name'.format(label_col)]], left_on='Class', right_on='USE_NAME', how='left')
    print(pixdf2['{}_name'.format(label_col)].value_counts())"""

"print(classification_params['pixdf'])\npixdf = pd.read_csv(classification_params['pixdf'])\n#pixdf = pixdf.dropna(inplace = True)\n#print(pixdf)\n#print(pixdf.isnull().any().any())\nprint('sample breakdown by {}:'.format(classification_params['lc_mod']))\nlabel_col, new_lut = get_class_col(classification_params['lc_mod'], lut)\nif '{}_name'.format(label_col) in pixdf.columns:\n    print(pixdf['{}_name'.format(label_col)].value_counts())\nelse:\n    pixdf2 = pixdf.merge(new_lut[['USE_NAME','{}'.format(label_col),'{}_name'.format(label_col)]], left_on='Class', right_on='USE_NAME', how='left')\n    print(pixdf2['{}_name'.format(label_col)].value_counts())"

## create rf model
this uses the multiclass RandomForestClassifier method from sklearn.ensemble (code is in ../LUCinSA_helpers/rf.py)

To use a different classification model, change 'lc_mod' in the parameters and rerun
current models = ('All' | 'trans_cats" | 'crop_nocrop' | 'crop_nocrop_medcrop' | 'crop_nocrop_medcrop_tree' | 'veg' | 'cropType' or 'single_X' (where X is any unique string in the USE_NAME column) for binary classification of X vs all else) 

In [585]:

rf0 = rf_model(classification_params['pixdf'],
         classification_params['local_dir'],
         classification_params['lc_mod'],
         classification_params['impmeth'],
         classification_params['ranhold'],
         classification_params['model_name'],
         lut)

class_col = LC25
there are 27819 pts in the full data set
there are 26127 sample points after removing those without clear class
There are 21818 training features
Out-of-bag score estimate: 0.628
Mean accuracy score: 0.648
getting confusion matrix based on LC25...
number of holdout pixels = 4309


## view confusion matrices
Note parameters: (pred_col, obs_col, lut, lc_mod_map, lc_mod_acc, print_cm=False, out_dir=None, model_name=None)
To print cm to csv file, change print_cm to True and provide an out_dir and model_Name

In [586]:
cm_cropNoCrop = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'crop_nocrop',True,classification_params['local_model_dir'],'Nov_default')
cm_cropType = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'cropType',True,classification_params['local_model_dir'],'Nov_default')
cm_veg = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'veg',False,classification_params['local_model_dir'],'Nov_default')
cm_all = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'All',False,classification_params['local_model_dir'],'Nov_default')
#cm_single = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],classification_params['lc_mod'],False,classification_params['local_model_dir'],None)

#print(cm_cropNoCrop)
#print(cm_cropType)
#print(cm_veg)
#print(cm_all)
#print(cm_single)

types = ["cropNoCrop", "cropType", "veg", "all"]
mats = [cm_cropNoCrop, cm_cropType, cm_veg, cm_all]

#saves the small cms to the /testing/{}_cms folder
index = 0
for i in types:
    mats[index].to_csv("/home/ryanashraf/LUCinSA_helpers/testing/{}_cms/{}_{}_{}.csv".format(i, mod_type, mod_name, i))
    index += 1

print("Done!")

getting confusion matrix based on LC2...
getting confusion matrix based on LC_crops...
getting confusion matrix based on LC5...
getting confusion matrix based on LC25...
Done!


## view variable importance
this can be computed via Impurity or Permutation method (see sklearn docs)  by setting impmeth in rf_model
The full list is stored in the model directory for further manipulation 

In [587]:
var_imp_path = os.path.join(classification_params['local_dir'],'VarImportance_{}.csv'.format(classification_params['model_name']))
var_imp = pd.read_csv(var_imp_path, names=['var','imp'], header=None)
## view 10 most important variables:
var_imp.sort_values('imp', ascending=False).head(10)

,var,imp
3,var_kndvi_amp_yr,0.053603
23,var_kndvi_minv_wet,0.050666
5,var_kndvi_sd_yr,0.048373
1,var_kndvi_maxv_yr,0.044190
6,var_kndvi_cv_yr,0.040981
10,var_kndvi_Apr_20,0.039788
24,var_kndvi_med_wet,0.037994
4,var_kndvi_avg_yr,0.037885
11,var_kndvi_Sep_20,0.037628
7,var_kndvi_Jan_20,0.035752


In [588]:


def small_acc(rf_mod, one = False, half = False):
    if one == True:
        one_ha = get_smallholder(rf_mod[1], one = True) # df for 1_ha smallholder ag
        one_ha_cropNoCrop = get_confusion_matrix(one_ha['pred'], one_ha['label'],lut,classification_params['lc_mod'],'crop_nocrop',True,classification_params['local_model_dir'],'Nov_default')
        return one_ha_cropNoCrop.head(1)["crop"]/one_ha_cropNoCrop.head(1)["All"]
    
    if half == True:
        half_ha = get_smallholder(rf_mod[1], half = True) # df for half_ha smallholder ag
        half_ha_cropNoCrop = get_confusion_matrix(half_ha['pred'], half_ha['label'],lut,classification_params['lc_mod'],'crop_nocrop',True,classification_params['local_model_dir'],'Nov_default')
        return half_ha_cropNoCrop.head(1)["crop"]/half_ha_cropNoCrop.head(1)["All"]



In [589]:
## take the cms and find take the averages of the UAs and PAs

# only run this section once to create the csvs and then comment out

"""
cropNoCrop_metrics = pd.DataFrame({"Model": ["{}_{}".format(mod_type, mod_name)],
                             "UA": [wave("/home/ryanashraf/LUCinSA_helpers/testing/cropNoCrop_cms/Max_base1000_cropNoCrop.csv", UA = True)],
                             "PA": [wave("/home/ryanashraf/LUCinSA_helpers/testing/cropNoCrop_cms/Max_base1000_cropNoCrop.csv", PA = True)],
                             "1_ha": [small_acc(rf0, one = True)[0]], #create a new function that uses the smallholder rf outputs in a useful manner
                             "half_ha": [small_acc(rf0, half = True)[0]],
                             "No. of Obs.": [len(pixdf["LC25_name_y"])]})
cropType_metrics = pd.DataFrame({"Model": ["{}_{}".format(mod_type, mod_name)],
                             "UA": [wave("/home/ryanashraf/LUCinSA_helpers/testing/cropType_cms/Max_base1000_cropType.csv", UA = True)],
                             "PA": [wave("/home/ryanashraf/LUCinSA_helpers/testing/cropType_cms/Max_base1000_cropType.csv", PA = True)],
                             "No. of Obs.": [len(pixdf["LC25_name_y"])]})
veg_metrics = pd.DataFrame({"Model": ["{}_{}".format(mod_type, mod_name)],
                             "UA": [wave("/home/ryanashraf/LUCinSA_helpers/testing/veg_cms/Max_base1000_veg.csv", UA = True)],
                             "PA": [wave("/home/ryanashraf/LUCinSA_helpers/testing/veg_cms/Max_base1000_veg.csv", PA = True)],
                             "No. of Obs.": [len(pixdf["LC25_name_y"])]})
all_metrics = pd.DataFrame({"Model": ["{}_{}".format(mod_type, mod_name)],
                             "UA": [wave("/home/ryanashraf/LUCinSA_helpers/testing/all_cms/Max_base1000_all.csv", UA = True)],
                             "PA": [wave("/home/ryanashraf/LUCinSA_helpers/testing/all_cms/Max_base1000_all.csv", PA = True)],
                             "No. of Obs.": [len(pixdf["LC25_name_y"])]})

print(cropNoCrop_metrics)
print(cropType_metrics)
print(veg_metrics)
print(all_metrics)

matrics = [cropNoCrop_metrics, cropType_metrics, veg_metrics, all_metrics]

#saves the cms to the /testing/metrics folder
index = 0
for i in types:
    matrics[index].to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/{}_metrics.csv".format(i))
    index += 1
"""


'\ncropNoCrop_metrics = pd.DataFrame({"Model": ["{}_{}".format(mod_type, mod_name)],\n                             "UA": [wave("/home/ryanashraf/LUCinSA_helpers/testing/cropNoCrop_cms/Max_base1000_cropNoCrop.csv", UA = True)],\n                             "PA": [wave("/home/ryanashraf/LUCinSA_helpers/testing/cropNoCrop_cms/Max_base1000_cropNoCrop.csv", PA = True)],\n                             "1_ha": [small_acc(rf0, one = True)[0]], #create a new function that uses the smallholder rf outputs in a useful manner\n                             "half_ha": [small_acc(rf0, half = True)[0]],\n                             "No. of Obs.": [len(pixdf["LC25_name_y"])]})\ncropType_metrics = pd.DataFrame({"Model": ["{}_{}".format(mod_type, mod_name)],\n                             "UA": [wave("/home/ryanashraf/LUCinSA_helpers/testing/cropType_cms/Max_base1000_cropType.csv", UA = True)],\n                             "PA": [wave("/home/ryanashraf/LUCinSA_helpers/testing/cropType_cms/Max_base1000_cr

In [590]:

# open the csvs and add a new row with the new data

def new_wave(cm_path, stored_path, model_name):
    stored = pd.read_csv(stored_path, index_col = 0)
    new_data = pd.DataFrame({"Model": [model_name],
                             "UA": [wave(cm_path, UA = True)],
                             "PA": [wave(cm_path, PA = True)],
                             "1_ha": [small_acc(rf0, one = True)[0]],
                             "half_ha": [small_acc(rf0, half = True)[0]],
                             "No. of Obs.": [len(pixdf["LC25_name_y"])]})
    #print(stored.reset_index(drop=True))
    #print(new_data.reset_index(drop=True))
    stored = pd.concat([stored.reset_index(drop = True), new_data.reset_index(drop = True)], ignore_index = True)
    return stored


In [591]:

types = ["cropNoCrop", "cropType", "veg", "all"]
mats = [cm_cropNoCrop, cm_cropType, cm_veg, cm_all]

#updates the metric csvs in /testing/metrics folder
index = 0
for i in types:
    mats[index] = new_wave("/home/ryanashraf/LUCinSA_helpers/testing/{}_cms/{}_{}_{}.csv".format(i, mod_type, mod_name, i), 
                "/home/ryanashraf/LUCinSA_helpers/testing/metrics/{}_metrics.csv".format(i),
                "{}_{}".format(mod_type, mod_name))
    mats[index].to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/{}_metrics.csv".format(i))
    print(mats[index])
    index += 1


getting confusion matrix based on LC2...
getting confusion matrix based on LC2...
                             Model        UA        PA      1_ha   half_ha  \
0                     Max_base1000  0.841363  0.838797  0.660714  0.660714   
1                 Max_ratios_rebal  0.931965  0.934273  0.869813  0.869813   
2                       Max_ratios  0.924233  0.950006  0.793103  0.793103   
3              Max_no_nbr_base1000  0.839394  0.836214  0.658163  0.658163   
4             Max_no_ndmi_base1000  0.839448  0.836781  0.656888  0.656888   
5          Max_no_nbrndmi_base1000  0.844764  0.841093  0.670918  0.670918   
6       Max_no_nbrndmi5-9_base1000  0.841731  0.837110  0.667092  0.667092   
7                Max_no_nbr_ratios  0.928859  0.950501  0.808429  0.808429   
8               Max_no_ndmi_ratios  0.924233  0.950006  0.793103  0.793103   
9            Max_no_nbrndmi_ratios  0.929521  0.953035  0.808429  0.808429   
10        Max_no_nbrndmi5-9_ratios  0.926380  0.949614  0.80

In [592]:
## use this to remove certain rows from the dfs that you do not want anymore

"""
types = ["cropNoCrop", "cropType", "veg", "all"]

delrow = 0
for i in types:
    mat = pd.read_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/{}_metrics.csv".format(i), index_col = 0)
    
    mat = mat.drop([delrow])
    print(mat)
    mat.to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/{}_metrics.csv".format(i))
"""


'\ntypes = ["cropNoCrop", "cropType", "veg", "all"]\n\ndelrow = 0\nfor i in types:\n    mat = pd.read_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/{}_metrics.csv".format(i), index_col = 0)\n    \n    mat = mat.drop([delrow])\n    print(mat)\n    mat.to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/{}_metrics.csv".format(i))\n'

In [593]:
def overall_wave(CNC_metrics, CT_path, model_name):
    
    # iterate through rows of the CNC_metrics dataframe
    CNC = pd.read_csv(CNC_metrics, index_col = 0)
    for index, row in CNC.iterrows():
        #print(row["Model"])
        if row["Model"] == model_name:
            CNC_partial = row

    # now deal with the CT matrix
    CT = pd.read_csv(CT_path, index_col = 0)
    CT_partial = [wave(CT_path, UA = True, weights = "CT"), wave(CT_path, PA = True, weights = "CT")]
    
    CNC_partial = [CNC_partial["UA"], CNC_partial["PA"], CNC_partial["1_ha"], CNC_partial["half_ha"]]
    #print(CNC_partial)
    #print(CT_partial)

    overall_metrics = pd.DataFrame({"Model": ["{}_{}".format(mod_type, mod_name)],
                             "UA": [(2 * CNC_partial[0] + CT_partial[0])/3],
                             "PA": [(2 * CNC_partial[1] + CT_partial[1])/3],
                             "1_ha": [CNC_partial[2]],
                             "half_ha": [CNC_partial[3]],
                             "No. of Obs.": [len(pixdf["LC25_name_y"])]})
    return overall_metrics


In [594]:
print(overall_wave("/home/ryanashraf/LUCinSA_helpers/testing/metrics/cropNoCrop_metrics.csv", 
                        "/home/ryanashraf/LUCinSA_helpers/testing/cropType_cms/{}_{}_cropType.csv".format(mod_type, mod_name), 
                        "{}_{}".format(mod_type, mod_name)))

                            Model        UA       PA      1_ha   half_ha  \
0  Max_nopp_nnweg5-9_ratios_rebal  0.798007  0.81414  0.806017  0.806017   

   No. of Obs.  
0        27819  


In [595]:
## create the baseline for the new overall metrics, once again comment this code out after running it for the first time

"""
overall = overall_wave("/home/ryanashraf/LUCinSA_helpers/testing/metrics/cropNoCrop_metrics.csv", 
             "/home/ryanashraf/LUCinSA_helpers/testing/cropType_cms/Max_ratios_cropType.csv", 
             "Max_ratios")
overall.to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv")
"""


'\noverall = overall_wave("/home/ryanashraf/LUCinSA_helpers/testing/metrics/cropNoCrop_metrics.csv", \n             "/home/ryanashraf/LUCinSA_helpers/testing/cropType_cms/Max_ratios_cropType.csv", \n             "Max_ratios")\noverall.to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv")\n'

In [596]:
## add new rows to the overall_metrics csv

stored = pd.read_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv", index_col = 0)
new_data = overall_wave("/home/ryanashraf/LUCinSA_helpers/testing/metrics/cropNoCrop_metrics.csv", 
                        "/home/ryanashraf/LUCinSA_helpers/testing/cropType_cms/{}_{}_cropType.csv".format(mod_type, mod_name), 
                        "{}_{}".format(mod_type, mod_name))
#print(stored.reset_index(drop=True))
#print(new_data.reset_index(drop=True))
stored = pd.concat([stored.reset_index(drop = True), new_data.reset_index(drop = True)], ignore_index = True)
print(stored)
stored.to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv")


                             Model        UA        PA      1_ha   half_ha  \
0                     Max_base1000  0.717750  0.755620  0.660714  0.660714   
1                 Max_ratios_rebal  0.847963  0.900477  0.869813  0.869813   
2                       Max_ratios  0.803116  0.898718  0.793103  0.793103   
3              Max_no_nbr_base1000  0.717922  0.750971  0.658163  0.658163   
4             Max_no_ndmi_base1000  0.716991  0.745947  0.656888  0.656888   
5          Max_no_nbrndmi_base1000  0.722365  0.750158  0.670918  0.670918   
6       Max_no_nbrndmi5-9_base1000  0.718525  0.749544  0.667092  0.667092   
7                Max_no_nbr_ratios  0.809394  0.897785  0.808429  0.808429   
8               Max_no_ndmi_ratios  0.804505  0.894229  0.793103  0.793103   
9            Max_no_nbrndmi_ratios  0.806911  0.897817  0.808429  0.808429   
10        Max_no_nbrndmi5-9_ratios  0.825993  0.902070  0.800766  0.800766   
11         Max_no_nbr_ratios_rebal  0.845440  0.900637  0.864627

In [597]:
## use this to remove certain rows from the df that you do not want anymore

"""
delrow = 16
mat = pd.read_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv", index_col = 0)
mat = mat.drop([delrow])
print(mat)
mat.to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv")
"""


'\ndelrow = 16\nmat = pd.read_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv", index_col = 0)\nmat = mat.drop([delrow])\nprint(mat)\nmat.to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv")\n'

## Make some tables that organize the data cleanly

In [598]:
## Organize all of the models that exclude poly_pred variables
stored = pd.read_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv", index_col = 0)

tab = []
for i in stored.iterrows():
    if i[1]["Model"] == "Max_{}".format(mod_name):
        tab.append(i[1])
    if i[1]["Model"].startswith("Max_no_pp_"):
        tab.append(i[1])

tab = pd.DataFrame(tab)
print(tab)


                      Model        UA        PA      1_ha   half_ha  \
1          Max_ratios_rebal  0.847963  0.900477  0.869813  0.869813   
19     Max_no_pp_e_base1000  0.709269  0.746673  0.639031  0.639031   
20    Max_no_pp_ed_base1000  0.692420  0.734790  0.598214  0.598214   
21   Max_no_pp_eda_base1000  0.694991  0.731874  0.603316  0.603316   
22  Max_no_pp_edaA_base1000  0.694595  0.740863  0.596939  0.596939   
23   Max_no_pp_all_base1000  0.684415  0.725504  0.573980  0.573980   

    No. of Obs.  
1         27819  
19        16853  
20        16853  
21        16853  
22        16853  
23        16853  


In [599]:
## 




In [600]:
## some cool graphs of the data you could have compiled








## To save an html copy of this notebook with all outputs:
(these two cells should be last in notebook)

In [601]:
%%script echo skipping
### comment out above line and run this cell to print output as html

out_name = str(basic_config['country']+'6c_RandomFoest_ModelComparisons'+'_model'+str(classification_params['model_name'])+'basic_config['filter_yr'])
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --ExecutePreprocessor.store_widget_state=True --output=$out_name 6b_RandomFoest_ModelComparisons.ipynb

skipping
